<a href="https://colab.research.google.com/github/belanatal/PosPUCRio/blob/main/MVPSprintIIb_IsabelaNatal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PUC Rio: Pós Graduação em Ciência de Dados e Analytics**

**MVP Sprint II: Machine Learning & Analytics (40530010056_20230_01)**
**- Parte b**

Aluna: Isabela Fernanda Natal Batista Abreu Gomes

Julho/2023

## Seção I: Introdução
**Classificador de imagens multiclasse - Paisagens Naturais**

**Contexto:** O presente trabalho tem como objetivo criar um modelo de visão computacional baseado em Aprendizado de Máquina Profundo que seja capaz de classificar uma imagem segundo uma das 6 seguintes categorias de paisagens: Prédio; Floresta; Montanha; Geleira; Rua; Mar. Os dados/imagens foram baixados da plataforma Kaggle (Para mais informações "Scene Classification": https://www.kaggle.com/datasets/nitishabharathi/scene-classification).

**Estrutura:** O notebook encontra-se dividido da seguinte forma:

- Importação das bibliotecas
- Acesso e tratamento dos dados que serão a entrada do modelo de deep learning
- Configuração do modelo de deep learning usando uma rede neural convolucional simples com Keras
- Treinamento do modelo de deep learning
- Execução do modelo de deep learning treinado
- Avaliação do modelo de deep learning
- Exportação do modelo de deep learning
- Teste do modelo exportado

### Importando as bibliotecas necessárias para executar o notebook
(Serão utilizadas as bibliotecas pandas e numpy, para a manipulação dos dados; matplotlib, para geração de gráficos; os, para manipulação de pastas e diretórios e bibliotecas voltadas para Machine e Deep Learning, tais como Keras, Tensor Flow e Scikit-Learn)

In [ ]:
!pip install tensorflow==2.13
!pip install keras

In [ ]:
!pip install kaggle

In [ ]:
# Primeiro bloco: Importação das bibliotecas e módulos
# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")
#
import os
import cv2 # Visão computacional para análise em imagens
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn.metrics as skm
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import Adam
from IPython.display import Image, display
from datetime import datetime
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#
#Para utilizar o upload e download de arquivos no colab:
from google.colab import drive
from google.colab import files

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/drive/MyDrive/Colab_Notebooks/Kaggle_API_Credentials/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download nitishabharathi/scene-classification

scene-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip scene-classification.zip

Archive:  scene-classification.zip
replace test_WyRytb0.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
if os.path.exists('/content/train-scene classification'):
  os.rename('../content/train-scene classification','../content/scene')

In [ ]:
if os.path.exists('/content/scene/train/*'):
  os.rename('../content/scene/train/*','../content/scene/Imagens/*')

In [ ]:
treino=pd.read_csv('../content/scene/train.csv',sep=',')
teste=pd.read_csv('../content/test_WyRytb0.csv',sep=',')

In [ ]:
treino.head()

,image_name,label
0,0.jpg,0
1,1.jpg,4
2,2.jpg,5
3,4.jpg,0
4,7.jpg,4


In [ ]:
# Verificando as dimensões dos DataFrames com as indicações das imagens correspondentes aos conjuntos de treino, teste
print(treino.shape,teste.shape)

(17034, 2) (7301, 1)


In [ ]:
# Verificando se os DataFrames contêm dados nulos
# Verify if the dataframe don't have NaN or null values
check_for_nan_treino = treino.isnull().values.any()
check_for_nan_teste = teste.isnull().values.any()

print(check_for_nan_treino)
print(check_for_nan_teste)

False
False


In [ ]:
# Criando a lista para armazenar os dados e selecionar a pasta da imagem a ser carregada
arquivos_com_label = []
labels = []
arquivos_teste = []
arquivos_teste_prediction = []
imagePath = "../content/scene/Imagens/"

## Seção II: Passos iniciais para tratamento dos dados

### Acessando e tratando os dados que serão a entrada do modelo de deep learning

In [ ]:
# Definindo o tamanho do batch e dimensão das imagens e a quantidade de épocas
batch_size = 32
img_height = 224
img_width = 224
num_classes = 6
epochs = 30

In [ ]:
# Associando as imagens ao dataset de treino
for i in treino.index:
    nameOfFile = treino['image_name'][i]
    if os.path.exists(imagePath+nameOfFile):
        image = mpimg.imread(imagePath+nameOfFile)
        if (len(image.shape)!=3): # Verify if the image is correct
            print("A imagem N°",i,' : ',nameOfFile," não está adequada")
        else :
            image = cv2.resize(image,(img_width,img_height)) # conjunto de treino sem imagens "expúrias"
            arquivos_com_label.append(image)
            labels.append(treino['label'][i])
print(len(arquivos_com_label))
print(len(labels))

17034
17034


In [ ]:
# Convertendo o dataset de treino em array
arquivos_com_label = np.array(arquivos_com_label)
labels = np.array(labels)
print(arquivos_com_label.shape,labels.shape)

(17034, 224, 224, 3) (17034,)


In [ ]:
# Associando as imagens ao dataset de teste
# As imagens do dataset de teste contêm apenas o nome. Os labels (categorias de paisagem) serão preditos pelo modelo
for i in teste.index:
    nameOfFile = teste['image_name'][i]
    if os.path.exists(imagePath+nameOfFile):
        image_teste = mpimg.imread(imagePath+nameOfFile)
        if (len(image.shape)!=3):
            print("A imagem N°",i,' : ',nameOfFile," não está adequada")
        else :
            image_teste = cv2.resize(image_teste,(img_width,img_height)) # conjunto de teste sem imagens "expúrias"
            arquivos_teste.append(image_teste)
print(len(arquivos_teste))

A imagem N° 2280  :  7606.jpg  não está adequada
7300


In [ ]:
# Convertendo o dataset de teste em array
arquivos_teste = np.array(arquivos_teste)
print(arquivos_teste.shape)

(7300, 224, 224, 3)


In [ ]:
# Definindo os caminhos de diretorios de treino, validação e teste
train_dir = '/content/scene/treino'
val_dir = '/content/scene/validacao'
test_dir = '/content/scene/teste'
treinamento_modelos_dir='/content/scene/trained_models'
if not os.path.exists(train_dir):
  os.makedirs(train_dir)
if not os.path.exists(val_dir):
  os.makedirs(val_dir)
if not os.path.exists(test_dir):
  os.makedirs(test_dir)
if not os.path.exists(treinamento_modelos_dir):
  os.makedirs(treinamento_modelos_dir)

## Seção III: Configuração do Modelo de Deep Learning

### Configuração do Modelo usando uma Rede Neural Convolucional (CNN) Simples

Utilizando a biblioteca Keras, será especificada uma camada convolucional 2D com 32 filtros (depois 64, 128 e 256) e função de ativação do tipo ReLU. Na sequência é adicionada uma camada softmax com o mesmo tipo de função de ativação.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 3, activation='relu', padding= "same", input_shape=(img_height,img_width,3)),
    keras.layers.MaxPooling2D((2)),
    keras.layers.Conv2D(128, 3, activation='relu', padding= "same"),
    keras.layers.Conv2D(128, 3, activation='relu', padding= "same"),
    keras.layers.MaxPooling2D((2)),
    keras.layers.Conv2D(256, 3, activation='relu', padding= "same"),
    keras.layers.Conv2D(256, 3, activation='relu', padding= "same"),
    keras.layers.Conv2D(256, 3, activation='relu', padding= "same"),
    keras.layers.MaxPooling2D((2)),
    keras.layers.Conv2D(512, 3, activation='relu', padding= "same"),
    keras.layers.Conv2D(512, 3, activation='relu', padding= "same"),
    keras.layers.Conv2D(512, 3, activation='relu', padding= "same"),
    keras.layers.MaxPooling2D((2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(num_classes, activation='softmax')
])

In [ ]:
# Resumindo o modelo que será utilizado
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 56, 56, 256)       2

### Treinamento do modelo de deep learning

Vamos considerar o modelo Adam para otimização. Como métrica para avaliação do modelo, será usado o critério de acurácia. Além disso, por se tratar de um problema de classificação com mais de 2 categorias (6, neste problema), a perda considerada será do tipo entropia cruzada.

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

### Separação das imagens com labels fornecidos em treino e validação
As imagens com categorias fornecidas conforme DataFrame treino serão divididas em treino de fato e validação

In [ ]:
# Criação dos datasets de treino e validação (10% do conjunto original de treino, isto é, aproximadamente 1700 imagens serão separados para validação do modelo)
X_train, X_val, y_train, y_val = train_test_split(arquivos_com_label,
                                                    labels,
                                                    test_size=0.10,
                                                    train_size=0.90,
                                                    random_state=7)

In [ ]:
# Avaliando os comprimentos dos datasets de treino e validação
print(X_train.shape,y_train.shape,X_val.shape,y_val.shape)
print(X_train.dtype,y_train.dtype,X_val.dtype,y_val.dtype)
# Alterando o tipo dos labels para inteiro (Lembrando que: 0-Edifício ; 1-Floresta; 2-Montanha; 3-Geleira; 4-Rua; 5-Mar)
y_train = y_train.astype(np.uint8)
y_val = y_val.astype(np.uint8)

(15330, 224, 224, 3) (15330,) (1704, 224, 224, 3) (1704,)
uint8 int64 uint8 int64


In [ ]:
print(X_train.dtype,y_train.dtype,X_val.dtype,y_val.dtype)

uint8 uint8 uint8 uint8


In [ ]:
# Confirmando que os datasets possuem quantidades representativas de cada uma das seis categorias
unique, counts = np.unique(y_train, return_counts=True)
print('Distribuição quantitativa dos labels no dataset de treino:')
print(dict(zip(unique, counts)))
unique, counts = np.unique(y_val, return_counts=True)
print('Distribuição quantitativa dos labels no dataset de validação:')
print(dict(zip(unique, counts)))

Distribuição quantitativa dos labels no dataset de treino:
{0: 2343, 1: 2498, 2: 2650, 3: 2732, 4: 2506, 5: 2601}
Distribuição quantitativa dos labels no dataset de validação:
{0: 285, 1: 247, 2: 307, 3: 305, 4: 278, 5: 282}


## Avaliar em 21/07

In [ ]:
# Visualizando algumas imagens do conjunto de treino
fig = plt.gcf()
fig.set_size_inches(3*4, 3*4)

# Visualizando apenas 12 imagens que têm categoria associada
for i in range(12):

    sample = np.random.choice(arquivos_com_label)
    sp = plt.subplot(3, 4, i + 1)
    sp.axis('Off')
    img = mpimg.imread(img_path)
    plt.imshow(img)
    plt.title(sample[:3])

plt.show()

In [ ]:
ntrain = X_train.shape[0]
nval = X_val.shape[0]

### Preparação dos dados
Será utilizada a classe "ImageDataGenerator", do TensorFlow, destinada a aumentar e pré-processar dados de imagem em tarefas de Aprendizado Profundo.

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
val_datagen = ImageDataGenerator()

In [ ]:
# Criação dos data generators
train_generator = train_datagen.flow(X_train, y_train,batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

In [ ]:
history = model.fit(train_generator, steps_per_epoch=ntrain//batch_size,epochs=epochs, validation_data=val_generator,validation_steps=nval//batch_size)

Epoch 1/30
479/479 [==============================] - 200s 370ms/step - loss: 1.6808 - accuracy: 0.2873 - val_loss: 1.4298 - val_accuracy: 0.4163
Epoch 2/30
479/479 [==============================] - 181s 377ms/step - loss: 1.4615 - accuracy: 0.4046 - val_loss: 1.2525 - val_accuracy: 0.5371
Epoch 3/30
479/479 [==============================] - 180s 374ms/step - loss: 1.3341 - accuracy: 0.4690 - val_loss: 1.1244 - val_accuracy: 0.6020
Epoch 4/30
479/479 [==============================] - 177s 368ms/step - loss: 1.2415 - accuracy: 0.5149 - val_loss: 0.9512 - val_accuracy: 0.6680
Epoch 5/30
479/479 [==============================] - 179s 373ms/step - loss: 1.1421 - accuracy: 0.5553 - val_loss: 0.8252 - val_accuracy: 0.7134
Epoch 6/30
479/479 [==============================] - 179s 373ms/step - loss: 1.0795 - accuracy: 0.5906 - val_loss: 0.8618 - val_accuracy: 0.6798
Epoch 7/30
479/479 [==============================] - 181s 378ms/step - loss: 1.0005 - accuracy: 0.6291 - val_loss: 0.7383 -

## Seção IV: Execução do Modelo de Deep Learning
Nesta seção, o modelo treinado será aplicado em cada uma das imagens do dataset de teste, indicando a qual das 6 classes  (Edificio; Floresta; Montanha; Geleira; Rua; Mar) pertence.

##CONTINUAR DAQUI EM 21/07

In [ ]:
parent_dir = '../content/scene/teste'
count_images = 0
class_names = ['Edificio',
               'Floresta',
               'Montanha',
               'Geleira',
               'Rua',
               'Mar']
y_pred = list()
y_true = list()

In [ ]:
# Associando as imagens ao dataset de teste
# As imagens do dataset de teste contêm apenas o nome. Os labels (categorias de paisagem) serão preditos pelo modelo
for i in teste.index:
    nameOfFile = teste['image_name'][i]
    if os.path.exists(imagePath+nameOfFile):
        image = mpimg.imread(imagePath+nameOfFile)
        if (len(image.shape)!=3):
            print("A imagem N°",i,' : ',nameOfFile," não está adequada")
        else :
            image = cv2.resize(image,(img_width,img_height)) # conjunto de teste sem imagens "expúrias"
            # Previsão
            prediction = model.predict(image)
            arquivos_teste_prediction.append(prediction)
            arquivos_teste.append(image)
print(len(arquivos_teste))
print(len(arquivos_teste_prediction))

In [ ]:
# Percorrendo a pasta onde estão salvas as imagens de teste (dividida em diretórios, subdiretórios e arquivos)

for subdir, dirs, files in os.walk(parent_dir):
  for file in files:
    if file.endswith('.png') or file.endswith('.jpg'):  # Considerando que as imagens são de extensão .png ou .jpg
#
      count_images+=1
      split_path = os.path.join(subdir, file).split('/')
      label = split_path[3]
      y_true.append(label)
      img_path = os.path.join(subdir, file)
      display(Image(filename=img_path, width=300))
      img = image.load_img(img_path, target_size=(img_height, img_width))
      figura = image.img_to_array(img)
      figura = np.expand_dims(figura, axis=0)
      figura = figura.astype('float32') / 255.0
#
      # Previsão
      prediction = model.predict(figura)
#
      # Printando as saídas do modelo
      predicted_class = np.argmax(prediction[0])
      probability = prediction[0][predicted_class]
      y_pred.append(class_names[predicted_class])
      print("Categoria Real:", label)
      print("Categoria Prevista:", class_names[predicted_class])
      print("Probabilidade indicada para a categoria prevista:", probability)
      print("\n")

## Seção V: Avaliação do Modelo



In [ ]:
# Calculando as métricas para avaliação do modelo
acuracia = skm.accuracy_score(y_true, y_pred)
precisao = skm.precision_score(y_true, y_pred, average='weighted')
recall = skm.recall_score(y_true, y_pred, average='weighted')
f1score = skm.f1_score(y_true, y_pred, average='weighted')

print("Acurácia: ", acuracia)
print("Precisão: ", precisao)
print("Recall: ", recall)
print("F-Score: ", f1score)

### Matriz de Confusão

In [ ]:
cnf_matrix = confusion_matrix(y_true, y_pred, labels=class_names)
np.set_printoptions(precisao=2)

plt.figure()
plot_confusion_matrix(cnf_matrix,
                      classes=['Edificio',
                               'Floresta',
                               'Montanha',
                               'Geleira',
                               'Rua',
                               'Mar'],
                      normalize= False,
                      title='Matriz real x predição')

### Exportação do Modelo

In [ ]:
# Vamos salvar o modelo treinado para aplicação futura

# obtendo a data e hora atual
now = datetime.now()

# Definição do formato
format = '%Y-%m-%dT%H%M'

# Converter a data e hora em uma string com o formato especificado
formatted_datetime = now.strftime(format)

path_model = treinamento_modelos_dir

name_model = 'trained_model_' + formatted_datetime + '.h5'

# salvando o modelo
model.save("%s/%s" % (path_model, name_model))
print("Modelo salvo com o nome: ", name_model)